# Introduction to Akori Web Proyect Dataset Analysis

## Contents
* [Abstract](#abstract)
* [Simple Heatmap](#simple_heatmap)
* [Priority Map](#priority_map)
* [Salience Map](#salience_map)
	- [Pre-trained Performance](#pretrained)
	- [Trained Performance](#trained)
* [Spacial Bias](#spacial_bias)
* [Structural Bias](#struct_bias)

<a id='abstract'></a>
## Abstract
When the cognitive mechanisms that affect attention in the human brain are studied, the eye-movement behaviour is a good source of information to research it's process given that the visual pathways in the brain have been amply studied. When characterizing which processes influence attentional behaviour, these can be represented through what is called a salience model, where three principal sub-processes are often represented: bottom-up influence contains elements which relate to pre-attentional stimulus, inherently present in the image. Top-down influence is the container of attentional influences that are considered high-level processing by the brain, recruiting many cortical areas and being task-dependant. Finally, spacial inhibitory influence is needed which rewards the exploring of new areas in the visual field. Bottom-up influence has been the most studied because of it's direct relation to visual cortical areas that process physical patterns, such as colour, intensity of light, orientation, etc. Leaving top-down influence and inhibitory patterns as a compliment to encapsulate behaviours which are not defined yet and explained as a bias. This approach doesn't allow a real-life understanding of many human tasks and is only effective for very basic ones; Given a certain task, how can we distinguish possible top-down channels which are more relevant when characterizing eye-movement and thus attention? Answering this question could achieve a better mathematical model. In this study a method is proposed to separate top-down influence channels specifically using a human free-viewing experiment in a web-page environment. First, given humans are highly trained by every-day life in web-exploring, we expected a high top-down bias in their behaviour (poor prediction with bottom-up models like Itti-Koch). Second, we expected to discern which top-down patterns are influencing the most in this task in particular. Finally, a mathematical exploratory method to apply this work to other tasks is established, which allows to further develop theory in how both bottom-up and top-down patterns interact and is processed in the human brain.(**TODO: acortar y agregar reporte de resultados de modelo de saliencia)

## Dataset used
The Akori-web dataset is a collection of 80 subjects recorded at Pedro Maldonado's Neurosistemas Laboratory (neurosistemas.cl) from Universidad de Chile. Subjects' age ranged from (**TODO: completar info del dataset y postear imagen del protocolo experimental) 

<a id='simple_heatmap'></a>
## Making a simple heatmap
First, we explore what subjects eye movement looks like with a heatmap visualization. Given a certain image, we'll make heatmap for all subjects, to achieve this we need a function that loads a subject from the dataset as a fixation map and then apply a gaussian filtered version on top of the webpage. We saved these functions in our *scripts/* folder as they can be used many times in this notebook.

In [ ]:
import os
import matplotlib.pyplot as plt
import scripts.data_loader as loader


# Show the heatmap inline
%matplotlib inline
plt.rcParams['figure.dpi'] = 500  # image quality
plt.rcParams['figure.figsize'] = [10, 5]  # image size

page = 'Banco de Chile'  # page name
number = 1  # page number
sigma = 30  # guassian parameter
ignored = 2  # ignore first n-fixations
transparency = 0.7  # of heatmap overlay

# Make a list of subjects
dataset_dir = os.listdir('dataset/') 
    
# Make initial fixmap with first subject
total_fixmap, webpage = loader.csv_fixmap(dataset_dir, page, number, ignore=ignored, map='time', norm=True)

# Now make heatmap
total_heatmap = loader.make_heatmap(total_fixmap, sigma)

# Visualize results
plt.imshow(webpage)
plt.imshow(total_heatmap, alpha=transparency)
plt.show()


<a id='priority_heatmap'></a>
## Making a priority heatmap

A heatmap is good as a qualitative representation of what subjects look most, but we could also be interested in what subject looks first too. To make a simple representation of what areas in the image have more priority to users we'll make a heatmap but using fixation order in the fix-map in replacement of fixation duration:

In [ ]:
import os
import matplotlib.pyplot as plt
import scripts.data_loader as loader


# Show the heatmap inline
%matplotlib inline
plt.rcParams['figure.dpi'] = 500  # image quality
plt.rcParams['figure.figsize'] = [10, 5]  # image size

page = 'Banco de Chile'  # page name
number = 1  # page number
sigma = 30  # guassian parameter
ignored = 2  # ignore first n-fixations
transparency = 0.7  # of heatmap overlay

# Make a list of subjects
dataset_dir = os.listdir('dataset/') 
    
# Make initial fixmap with first subject
total_fixmap, webpage = loader.csv_fixmap(dataset_dir, page, number, ignore=ignored, map='priority', norm=True)

# Now make heatmap
total_heatmap = loader.make_heatmap(total_fixmap, sigma)

# Visualize results
plt.imshow(webpage)
plt.imshow(total_heatmap, alpha=transparency)
plt.show()


<a id='salience_map'></a>
## Computing Salience Map
The final achievement of this project is predicting user's ocular behaviour when exploring web-pages, to achieve the best possible result, we believe fusing the best performance algorithm's from the engineering field with the expertise in attentional mechanisms from neuroscience will achieve the best possible result. To prove this we start by using a pre-trained model with non-web database *TODO: agregar explicación del benchmark del MIT*, followed by a trained method of the same algorithm and a final optimization to the algorithm to specialize it to web-page environments. For each model we use NSS and AUC-Judd to quantify performance.

<a id='pretrained'></a>
### Pre-trained SAM-salience model vs All database

For a trained model we already computed results in */predictions_sam/* folder just to save time and processing power (TODO: Migrate this process to my free Amazon Web Service)

In [1]:
from scripts.salience_metrics import nss
from PIL import Image
import os
import scripts.data_loader as loader
import numpy as np

scores = []

ignored = 2  # ignore first n-fixations
page_set = os.listdir('pages/')
subject_set = os.listdir('dataset/')

for a_page in page_set:  # we compute all five sub-pages
    print("Processing page " + a_page)
    sub_page_set = os.listdir("pages/{page}/".format(page=a_page))
    for number in range(5):
        # Make fixmap
        total_fixmap, webpage = loader.csv_fixmap(subject_set, a_page, number+1, ignore=ignored, map='time', norm=True)
        # Load pre-trained salience
        image_id = "{page} {num}_salience.jpg".format(page=a_page, num=number+1)
        salmap = np.array(Image.open("predictions_sam/{page} {num}_salience.jpg".format(page=a_page, num=number+1)))
        salmap_normalized = salmap/salmap.max()  # normalize saliency map
        # Compute score
        scores.append(nss(salmap_normalized, total_fixmap))
print(np.mean(scores))

Processing page Adidas


Processing page Banco BBVA


Processing page Banco de Chile


Processing page Banco Estado


Processing page Banco Santander


...

Processing page Mercado Libre


Processing page Movistar


Processing page Ripley


Processing page VTR


1.2844139086121593


<a id='spacial_bias'></a>
## Exploring Spacial Bias in Behaviour
From priority map computation we suspect subjects may have a spacial bias (top-left) when exploring a web-page, if this is true, the average priority map should be biased towards top-left independant of webpage, for this reason we compute a grand average heatmap of the whole database. (*TODO: In work*)

<a id='struct_bias'></a>
## Exploring Structural Bias in Behaviour

Subjects may show a bias towards certain objects types in the image (i.e: pictures over text), we want to explore this phenomena to be able to refine or specialize our salience map towards web-page environments. (TODO: *After spacial bias*)
